0.0 Importing Packages

In [1]:
#Importing packages
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import math as m
from sklearn.model_selection import train_test_split
import random as rand
import csv
from google.colab import drive
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import random
from torch import optim
import tensorflow as tf
from tensorflow import keras
from keras import layers, datasets, models, optimizers
from keras.layers import Dense
from keras.models import Sequential
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib widget

import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from google.colab import output
output.enable_custom_widget_manager()


1.0 Mounting the Drive and creating the data set

In [2]:
drive.mount('/drive')

Mounted at /drive


In [3]:


# open the file in the write mode
f = open('/drive/My Drive/ME 780/TestDataset.csv', 'w')

# create the csv writer
writer = csv.writer(f)

#Grabbing points for x1 and x2
x1 = np.linspace(-2, 2, num=16, endpoint=False)
x2 = np.linspace(-2, 2, num=16, endpoint=False)



#Replacing 0 to avoid dividing by zero
x1z = np.where(x1 == 0.)
x2z = np.where(x2 == 0.)

x1[x1z] = 0.001
x2[x2z] = 0.001




writer.writerow(['X1', 'X2', 'Y1', 'Y2'])

c1 = 0
c2 = 0
for i in x1:
  for j in x2:
    rho = i**2 + j**2
    y1 = i*np.exp(-rho)
    term1 = np.cos(rho)
    y2 = np.divide(term1, j)
    row = [i, j, y1, y2]
    # write a row to the csv file
    writer.writerow(row)
    c2 +=1
  c2 = 0
  c1 +=1
f.close()


In [ ]:


# open the file in the write mode
f = open('/drive/My Drive/ME 780/RandomDataset4.csv', 'w')

# create the csv writer
writer = csv.writer(f)


#Grabbing points for x1 and x2
x1 = np.linspace(-2,2,num=100, endpoint = False)
x2 = np.linspace(-2,2,num=100, endpoint = False)


#Replacing 0 to avoid dividing by zero
x1z = np.where(x1 == 0.)
x2z = np.where(x2 == 0.)

x1[x1z] = 0.001
x2[x2z] = 0.001




writer.writerow(['X1', 'X2', 'Y1', 'Y2'])

c1 = 0
c2 = 0
while c1< 32:
  while c2 < 32:
    x_1 = random.choice(x1)
    x_2 = random.choice(x2)
    rho = x_1**2 + x_2**2
    y1 = x_1*np.exp(-rho)
    term1 = np.cos(rho)
    y2 = np.divide(term1, x_2)
    row = [x_1, x_2, y1, y2]
    # write a row to the csv file
    writer.writerow(row)
    c2 +=1
  c2 = 0
  c1 +=1
f.close()




2.0 Reading from the data set

In [3]:
dfTest = pd.read_csv('/drive/My Drive/ME 780/TestDataset.csv')


df = pd.read_csv('/drive/My Drive/ME 780/RandomDataset4.csv')



3.0 Attempting to Plot the resulting Data set

In [ ]:
def y1(x1, x2):
  return x1*np.exp(-(x1**2 + x2**2))

def y2(x1,x2):
  rho = x1**2 + x2**2
  term1 = np.cos(rho)
  return np.cos(rho)/x2

x1 = dfTest["X1"]
x2 = dfTest["X2"]



arrx1 = x1.values
arrx2 = x2.values


newx1 = arrx1.reshape(16,16)
newx2 = arrx2.reshape(16,16)


ex1, ex2 = np.meshgrid(newx1, newx2)


y1 = y1(ex1,ex2)
y2 = y2(ex1,ex2)


fig=plt.figure()




fig = plt.figure()
ax = fig.gca(projection='3d')
surf1 = ax.plot_surface(ex1, ex2, y1)
plt.show()


fig = plt.figure()
ax = fig.gca(projection='3d')
surf2 = ax.plot_surface(ex1, ex2, y2)
plt.show()




4.0 Training the model

In [ ]:
#Onto the neural network part of this
x1 = df["X1"]
x2 = df["X2"]
arrx1 = x1.values
arrx2 = x2.values


# copy the data
df_min_max_scaled = df.copy()
  
# apply normalization techniques by Column 1
column1 = "X1"
df_min_max_scaled[column1] = (df_min_max_scaled[column1] - df_min_max_scaled[column1].min()) / (df_min_max_scaled[column1].max() - df_min_max_scaled[column1].min())  
column2 = "X2"
df_min_max_scaled[column2] = (df_min_max_scaled[column2] - df_min_max_scaled[column2].min()) / (df_min_max_scaled[column2].max() - df_min_max_scaled[column2].min())      
column3 = "Y1"
df_min_max_scaled[column3] = (df_min_max_scaled[column3] - df_min_max_scaled[column3].min()) / (df_min_max_scaled[column3].max() - df_min_max_scaled[column3].min())   
column4 = "Y2"
df_min_max_scaled[column4] = (df_min_max_scaled[column4] - df_min_max_scaled[column4].min()) / (df_min_max_scaled[column4].max() - df_min_max_scaled[column4].min())  
  

# copy the data
df_min_max_scaled_Test = dfTest.copy()
  
# apply normalization techniques by Column 1
column1 = "X1"
df_min_max_scaled_Test[column1] = (df_min_max_scaled[column1] - df_min_max_scaled[column1].min()) / (df_min_max_scaled[column1].max() - df_min_max_scaled[column1].min())  
column2 = "X2"
df_min_max_scaled_Test[column2] = (df_min_max_scaled[column2] - df_min_max_scaled[column2].min()) / (df_min_max_scaled[column2].max() - df_min_max_scaled[column2].min())      
column3 = "Y1"
df_min_max_scaled_Test[column3] = (df_min_max_scaled[column3] - df_min_max_scaled[column3].min()) / (df_min_max_scaled[column3].max() - df_min_max_scaled[column3].min())   
column4 = "Y2"
df_min_max_scaled_Test[column4] = (df_min_max_scaled[column4] - df_min_max_scaled[column4].min()) / (df_min_max_scaled[column4].max() - df_min_max_scaled[column4].min())  
  


X_train = df_min_max_scaled[["X1", "X2"]]
Y_train = df_min_max_scaled[["Y1", "Y2"]]

X_test = df_min_max_scaled_Test[["X1", "X2"]]
Y_test = df_min_max_scaled_Test[["Y1", "Y2"]]



#Defining layer components
inputs = keras.Input(shape=(2,))

dense = layers.Dense(64, activation="relu")
x = dense(inputs)

#Splitting the final layer into two for activation function adjustment
y1_pred = layers.Dense(1, name="Y1", activation=keras.activations.linear, bias_initializer='zeros' )(x)

y2_pred = layers.Dense(1, name="Y2", activation=keras.activations.linear, bias_initializer='zeros' )(x)


# Creating the model and indicating inputs and outputs
model = keras.Model(
    inputs=inputs,
    outputs=[y1_pred,y2_pred],
)
model.summary()

#Setting some hyperparameters
tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.4,
    name='SGD',
)

tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)


#Setting optimizers and loss functions
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss={
        "Y1": keras.losses.MeanSquaredError(),
        "Y2": keras.losses.MeanSquaredError(),
    },
    loss_weights={"Y1": 1.0, "Y2": 0.1},
)


#training the model and exporting insights
history = model.fit(X_train, Y_train,
    epochs=50,
)










5.0 Evaluating the Model


In [ ]:
plt.plot(history.history['loss'])
plt.show()

plt.plot(history.history['Y1_loss'])
plt.show()


plt.plot(history.history['Y2_loss'])
plt.show()

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(
    x=X_test,
    y=Y_test,
    batch_size=None,
    verbose='auto',
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
)

y1_pred, y2_pred = model.predict(X_test)

6.0 Plotting the Prediction


In [ ]:
def y1(x1, x2):
  return x1*np.exp(-(x1**2 + x2**2))

def y2(x1,x2):
  rho = x1**2 + x2**2
  term1 = np.cos(rho)
  return np.cos(rho)/x2

# I think i need to unnormalize
x1 = X_test["X1"]
x2 = X_test["X2"]
y1_test = Y_test["Y1"]
y2_test = Y_test["Y2"]

#y1_pred, y2_pred = model.predict(X_test)

arrx1 = x1.values
arrx2 = x2.values
arry1 = y1_test.values
arry2 = y2_test.values
reshapeto = len(x1)

newx1 = arrx1.reshape(reshapeto, 1)
newx2 = arrx2.reshape(reshapeto, 1)


nnewx1 = newx1.reshape(16,16)
nnewx2 = newx2.reshape(16,16)
newy1 = arry1.reshape(16,16)
newy2 = arry2.reshape(16,16)

newy1_pred = y1_pred.reshape(16,16)
newy2_pred = y2_pred.reshape(16,16)

ex1, ex2 = np.meshgrid(newx1, newx2)

y1 = y1(nnewx1,nnewx2)
y2 = y2(nnewx1,nnewx2)

why1 = np.empty([256,256])
why2 = np.empty([256,256])
input = np.empty([1,2])


fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.legend(loc='upper left')
#surf1 = ax.plot_surface(nnewx1, nnewx2, y1, label="Test data")
ax.scatter(newx1, newx2, y1_pred, label='Predicted')
ax.scatter(newx1, newx2, newy1, label = "Test data")
ax.set_xlabel('X1 Label')
ax.set_ylabel('X2 Label')
ax.set_zlabel('Y1 Label')

plt.show()


fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.legend(loc='upper left')
#surf1 = ax.plot_surface(nnewx1, nnewx2, y1, label="Test data")
ax.scatter(newx1, newx2, y2_pred, label='Predicted')
ax.scatter(newx1, newx2, newy2, label = "Test data")
ax.set_xlabel('X1 Label')
ax.set_ylabel('X2 Label')
ax.set_zlabel('Y2 Label')

plt.show()







